## 이거 밑에 block 빼서 쓰세요 미안!

In [1]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('cuda')
else:
    device = torch.device('cpu')
    print('cpu')

cuda


In [2]:
import numpy as np
f = open('live_record(1.567G, 50MHz, 25dB, 5M)_230513_2013_102s_SDR_검증.bin', 'rb')
data = np.fromfile(f, dtype=np.int16, count = int(50e6*10*2) + 4000)
f.close()


# 오류로 인한 값들 삭제
sig = []
signal_data = []

# 시간 복잡도를 줄인 배열 요소 삭제 알고리즘
for i in range(2,len(data),500002):
# i = 2, 500004, 1000006, ... (첫 두 개의 값도 spike) 
    frac = data[i:i+500000] # 50만 개씩 slicing
    real = frac[::2]
    imag = frac[1::2]
    signal_data.append(real + 1j * imag) # 자른 값을 결과에 추가
    
# 결과 sig 배열을 1차원으로 변경
signal_data = np.hstack(signal_data)


In [3]:

from scipy.signal import correlate
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import deque
from tqdm import tqdm  

# 각 위성의 G2 Resister에서 Code phase Selection을 정의.
sats = [(1, 5), (2, 6), (3, 7), (4, 8), (0, 8), (1, 9), (0, 7), (1, 8), (2, 9), (1, 2),
            (2, 3), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (0, 3), (1, 4), (2, 5), (3, 6),
            (4, 7), (5, 8), (0, 2), (3, 5), (4, 6), (5, 7), (6, 8), (7, 9), (0, 5), (1, 6),
            (2, 7), (3, 8), (4, 9), (3, 9), (0, 6), (1, 7), (3, 9)]

# 각 Resister에서 Feedback을 하기 위해 뽑는 tap의 위치
g1tap = [2,9]
g2tap = [1,2,5,7,8,9]

# 위성 번호에 따른 code 생성
def getCode(satsNum):
    
    g1 = deque(1 for i in range(10))
    g2 = deque(1 for i in range(10))
     
    # result
    result = []
    
    # Generating 1023 chips(One C/A sequence)
    for i in range(1023):
		    
		    # Append code to result
        val = (g1[9] + g2[satsNum[0]] + g2[satsNum[1]]) % 2
        result.append(val)
        
        # Shift g1 Resister
        g1[9] = sum(g1[i] for i in g1tap) % 2
        g1.rotate()
        
        # Shift g2 Resister
        g2[9] = sum(g2[i] for i in g2tap) % 2
        g2.rotate()
    
    # 0 => -1
    for n,i in enumerate(result):
            if i==0:
                result[n]=-1
        
    return result

# 코드 미리 만들어두기(시간복잡도 줄이려고)
codes = []
for i in range(37):
    codes.append(getCode(sats[i]))


In [4]:
# 50MHz Sampling, 1.023MHz C/A code chip rate
# 50/1.023 samples per chip 
# Sampling 주파수는 50MHz, C/A 코드의 코드 주파수는 1.023MHz이다. 그 결과 sampling을 수행하면 1023 chip이었던 code가 50000개가 되는 것이다.
# Chip 마다 48 또는 49개의 sample이 생성된다는 건데 (48.87)
# 이 값들이 반복되는 주기를 구해야 함. 즉 (50 / 1.023) * X가 정수가 되기 위한 X의 최솟값
# 구해보면 1023 chip 마다 정수배가 되니까 (50000 sample 째에서 정수가 됨)
# C/A 코드 한 주기(1023 Chip 안에서 정해짐)
# 이만큼만 반복문 돌려서 각 chip당 몇 sample 넣을 건지 계산하면 됨

sample_count = []
cur = 0

for i in range(1,1024):
    cur = 50 * i /1.023
    sample_count.append(int(round(cur - round(50 * (i-1) / 1.023))))

In [5]:
class MA_Filter:
    def __init__(self, signal, IF_freq):
        self.t = np.arange(0, 100000) / (50e6)
        base_carrier = np.exp(-1j * 2 * np.pi * IF_freq * self.t)
        self.BB_signal = signal * base_carrier
    
    def extract(self, doppler_freq):
        
        # Doppler frequency로 내리기
        e = np.exp(-1j * 2 * np.pi * doppler_freq * self.t)
        target = self.BB_signal * e
        
        # 절반 chip 만큼씩 step을 밟으면서, 1chip 길이의 합을 저장
        Ma = []
        pos = 0
        i = 0
        half = 1
        # 두 바퀴
        pos_lst = [0]
        
        doublecount = sample_count + sample_count
        try:
            while pos < 100000:
                if doublecount[i] == 48:
                    m = np.sum(target[pos:pos + 48])
                    Ma = np.append(Ma, m)
                    pos += 24
                elif doublecount[i] == 49:
                    if half == 1:
                        m = np.sum(target[pos:pos + 49])
                        Ma = np.append(Ma, m)
                        pos += 24
                    else:
                        m = np.sum(target[pos:pos + 49])
                        Ma = np.append(Ma, m)
                        pos += 25
                if half == -1:
                    i += 1
                half *= -1
                pos_lst.append(pos)
                
        except IndexError:
            np.append(Ma,np.sum(signal_data[99975:100024]))
          
        return Ma, pos_lst
    
              

In [6]:
def acquisition(signal, code_delay_range, doppler_freq_range, IF_freq, svNumber):
    
    frac_signal = signal[:100000]

    # 1ms 안에서 F(code delay & 주파수 offset '대략적으로' 찾기)
    max_corr = 0
    max_code_delay = 0
    max_doppler_freq = 0
    code_delay_sample = 0
    
    corr_matrix = np.zeros((len(code_delay_range), len(doppler_freq_range)))
    
    Ma = MA_Filter(frac_signal, IF_freq)

    code = codes[svNumber]
    for i, doppler_freq in enumerate(doppler_freq_range):
        
        target, pos_lst = Ma.extract(doppler_freq)
        
        for j, code_delay in enumerate(code_delay_range):
            
            # Target의 간격은 반 chip
            current_pos = target[j:j+2046:2]
            
            # Correlation
            cor = np.abs(np.sum(current_pos * code))
            peak = np.max(cor) 
            corr_matrix[j, i] = peak
            
            if peak > max_corr:
                max_corr = peak
                max_code_delay = code_delay
                max_doppler_freq = doppler_freq
                code_delay_sample = pos_lst[j]

    
    # Noise 계산
    mat = corr_matrix.flatten()
    noise_lst = np.percentile(mat, q=[99])
    noise = noise_lst[0]
  
    
    # Thershold : 1.9    
    if max_corr > noise * 1.9: # 보이는 SV인가?
        valid = True
    else:
        valid = False
    
    if valid:
        print('-'*40)
        print(f'SV {svNumber+1} Found.')
    else:
        #print(f'I think SV {svNumber+1} isn\'t here...')
        return max_code_delay, max_doppler_freq, corr_matrix, code_delay_sample, valid
    
    print('Estimated code delay :', max_code_delay)
    print('Estimated frequency offset :', max_doppler_freq)
    #print('Max correlation :', max_corr)
    
    return max_code_delay, max_doppler_freq, corr_matrix, code_delay_sample, valid

In [7]:
code_delay_range = np.arange(0, 1023, 0.5) # 50000 sample을 2046개로 나눔 (0.5chip 간격)
doppler_freq_range = np.linspace(-5000, 5000, 21)
IF_freq = 8.42e6

# 들어있는 위성 번호 : 5, 11, 13, 15, 20.
svNumber = 5

SVresult = []
# Perform acquisition
print('Checking...')
for i in range(37):
    estimated_code_delay, estimated_doppler_freq, corr_matrix, code_delay_sample, valid= acquisition(signal_data, code_delay_range, doppler_freq_range, IF_freq, i)
    if valid:
        SVresult.append([i, estimated_code_delay, estimated_doppler_freq, code_delay_sample])
svlst = [s[0]+ 1 for s in SVresult]
print('-'*40)
print('Found SV ', end = "")
for i in svlst:
    print(f'{i}, ', end ="")
print('.')


Checking...
----------------------------------------
SV 5 Found.
Estimated code delay : 164.0
Estimated frequency offset : 500.0
----------------------------------------
SV 11 Found.
Estimated code delay : 628.0
Estimated frequency offset : -2000.0
----------------------------------------
SV 13 Found.
Estimated code delay : 269.0
Estimated frequency offset : 1500.0
----------------------------------------
SV 15 Found.
Estimated code delay : 83.5
Estimated frequency offset : 3000.0
----------------------------------------
SV 20 Found.
Estimated code delay : 116.0
Estimated frequency offset : -500.0
----------------------------------------
SV 29 Found.
Estimated code delay : 970.5
Estimated frequency offset : -500.0
----------------------------------------
SV 30 Found.
Estimated code delay : 844.0
Estimated frequency offset : 0.0
----------------------------------------
Found SV 5, 11, 13, 15, 20, 29, 30, .


In [8]:
for s in SVresult:
    print(f"SV : {s[0]+1}, code delay offset : {s[1]}, doppler frequency offset : {int(s[2])}")

SV : 5, code delay offset : 164.0, doppler frequency offset : 500
SV : 11, code delay offset : 628.0, doppler frequency offset : -2000
SV : 13, code delay offset : 269.0, doppler frequency offset : 1500
SV : 15, code delay offset : 83.5, doppler frequency offset : 3000
SV : 20, code delay offset : 116.0, doppler frequency offset : -500
SV : 29, code delay offset : 970.5, doppler frequency offset : -500
SV : 30, code delay offset : 844.0, doppler frequency offset : 0


In [9]:
class Replica:
    def __init__(self, code_delay, doppler_freq, IF_freq, svNumber, t0): # t0 = 이전 carrier의 마지막 phase. Φ_n
        ca_code = np.array(codes[svNumber])
        ca_code = [chip for chip, cnt in zip(ca_code, sample_count) for _ in range(cnt)] # chip 당 해당하는 sample 개수 만큼 늘이기
        self.delayed_code = np.roll(ca_code, code_delay)
    
        self.t = np.arange(0,50000) / 50e6
        self.oversampled_signal = np.exp(1j * (2 * np.pi * (doppler_freq + IF_freq) * self.t + t0))   
                                                   
       # 다음 replica에서 사용할 Φ_(n+1) (누적), stay 판정 난 경우 가져올 값
        self.last = (2 * np.pi * (doppler_freq + IF_freq) * (50000/50e6) + t0) % (2*np.pi)   # angle
        self.last_early = (2 * np.pi * (doppler_freq + IF_freq) * (49999/50e6) + t0) % (2*np.pi)
        self.last_late = (2 * np.pi * (doppler_freq + IF_freq) * (50001/50e6) + t0) % (2*np.pi)
    

    def delay(self, delay):
        self.delayed_code = np.roll(self.delayed_code, delay)
    def signal(self):
        return self.delayed_code * self.oversampled_signal

In [10]:
class Freq_discriminator:
    def __init__(self):
        self.temp = 0
        
    def calculate(self, data, i):
        Ip1 = np.real(self.temp)
        Qp1 = np.imag(self.temp)
        Ip2 = np.real(data)
        Qp2 = np.imag(data)
        
        cross = Ip1 * Qp2 - Ip2 * Qp1
        dot = Ip1 * Ip2 + Qp1 * Qp2
        deg = np.arctan2(cross, dot)
    
        transition = False
        
        if abs(deg) > np.pi * 1.2 / 2:
            #print(f'bit transition at {i}, deg = {np.degrees(deg)}')
            val = deg - np.pi if deg > 0 else np.pi + deg
            transition = True
        else:
            val = deg
        #val = deg
            
        D = np.degrees(val) / (1e-3 * 360)
        
        self.temp = data
        
        return D, transition


In [15]:
def Tracking(svNumber, estimated_code_delay, estimated_doppler_freq, code_delay_sample):
    #Initialization
    current_code_delay = estimated_code_delay
    cur_pos = code_delay_sample
    current_doppler_freq = estimated_doppler_freq
    est_error_freq = 0
    current_phase_offset = 0
    stack_P = 0 # 초기에 시작할 Φ_0
    temp = 0 # phase offset 이전 값 저장


    b = 0 # bit boundary 감지 후 20개 모을 변수
    start_pos = 0 # boundary 시작 지점
    comp = [] # 20개 저장 후 bit decision


    # Plot 할 것들(초기값으로 채워주기)
    dop_freq_lst = [current_doppler_freq]
    phase_offset_lst = [0]
    phase_offset_lst_deg = [0]
    code_delay_lst = [current_code_delay] 
    cor_result = []
    demod_result = []
    bit = []

    # 복조 결과

    freq_disc = Freq_discriminator()

    cor_P = 0
    
    try:
        for i in range(9998):
            #bar.set_postfix({'현재 cor_P 값 :' : np.abs(cor_P)})
            frac_signal_P = signal_data[cur_pos : cur_pos + 50000]
            replica_P = Replica(0, current_doppler_freq, IF_freq, svNumber, stack_P)
            
            # C/A code correlation
            cor_P = correlate(frac_signal_P, replica_P.signal(), mode = 'valid')
            cfp = np.max(cor_P)
            cor_P = np.max(cor_P)* np.exp(-1j * current_phase_offset)
            
            # Late 계산을 위한 code delay (Half chip)            
            replica_P.delay(24)
            cor_L = correlate(frac_signal_P, replica_P.signal(), mode = 'valid')
            cor_L = np.max(cor_L)* np.exp(-1j * current_phase_offset)
            
            # Early 계산을 위한 code delay (Half chip)
            # 앞선 late에서 24를 당겨가서 여기선 한 chip 만큼 뒤로 당겨야 함
            replica_P.delay(-48)
            cor_E = correlate(frac_signal_P, replica_P.signal(), mode = 'valid')
            cor_E = np.max(cor_E)* np.exp(-1j * current_phase_offset)
            
            cor_result.append(cor_P)
            
            # FLL
            if i == 0:
                freq_disc.temp = cfp
            try: 
                est_error_freq, transition = freq_disc.calculate(cfp, i)
            except IndexError:
                est_error_freq = est_error_freq
            
            if i == 0:
                D_temp = est_error_freq
                
            #주파수 update (FLL Loop Filter) / 쓸 거면 주석을 떼시오
            # 1차
            #current_doppler_freq = current_doppler_freq * 0.7 + est_error_freq * 0.3
            # 2차
            #current_doppler_freq = current_doppler_freq + est_error_freq * 0.10119 + 0.002698 * (est_error_freq - D_temp)
            current_doppler_freq = current_doppler_freq + est_error_freq * 0.122396 + 0.005698 * (est_error_freq - D_temp)
                
            
            dop_freq_lst.append(current_doppler_freq)
            D_temp = est_error_freq
            
            
            # PLL
            angle = np.angle(cor_P)
            if np.abs(angle) > np.pi / 2:
                if angle < 0:
                    res = angle + np.pi
                else:
                    res = angle - np.pi
            else: # np.pi /2 보다 작을 때(1사분면, 4사분면)
                res = angle

            # PLL Loop Filter / 쓸 거면 주석을 떼시오
            # 1차
            #current_phase_offset = current_phase_offset * 0.7 + res * 0.3
            # 2차
            current_phase_offset = current_phase_offset + 0.0567 * res + 0.0754 * (res - temp)
            
            #stack_P = replica_P.last
            temp = res
            
            if res > np.pi :
                phase_offset_lst.append(res - 2 * np.pi)
                phase_offset_lst_deg.append(np.degrees(res - 2 * np.pi))
            else:
                phase_offset_lst.append(res)
                phase_offset_lst_deg.append(np.degrees(res))


            # DLL 
            IE = np.real(cor_E)
            QE = np.imag(cor_E)
            IL = np.real(cor_L)
            QL = np.imag(cor_L)
            e = ((IE**2 + QE**2) - (IL**2 + QL**2))/((IE**2 + QE**2) + (IL**2 + QL**2))
            
            if abs(e) <= 0.5:
                current_code_delay = current_code_delay
                cur_pos += 50000
                stack_P = replica_P.last
                
            else:    
                if e < -0.5:
                    current_code_delay += 1.023/50
                    cur_pos += 50001
                    stack_P = replica_P.last_late
                    
                elif e > 0.5:
                    current_code_delay -= 1.023/50
                    cur_pos += 49999
                    stack_P = replica_P.last_early
                    
                    
            code_delay_lst.append(current_code_delay)
        
            # Decision (Demodulation)
            if b == 0: # 아직 boundary 찾은 보고 없음
                if transition: # 최초 boundary 식별
                    b += 1
                    start_pos = i % 20 #가장 처음 지점부터 모을 것이기 때문에 그만큼 한번에 수집 (예 : 만약 46에서 찾았다면 bit 시작점은 6일 거임)
                    if i - start_pos >= 20: # 지금 현재 위치가 멀리 떨어져 있다면(1bit보다 더) (예 : 46 - 6 = 40이고, 앞선 2bit는 아직 안넣음. 밑은 넣는 작업)
                        for j in range((i - start_pos) // 20):
                            # 20개 수집
                            c = np.sum(cor_result[start_pos + 20 * j : start_pos + 20 *(j+1)])    
                            if np.real(c) >= 0:
                                bit.append(1)
                            else:
                                bit.append(0)
                    
            # 이후부턴 loop마다 1개 씩 수집, 20개가 차면 decision, bit 생성.        
            elif b == 1: # 20개 수집 시작
                comp.append(cor_P)
                if len(comp) == 20:
                    c = np.sum(comp)
                    if np.real(c) >= 0:
                        bit.append(1)
                    else:
                        bit.append(0)
                    # 리스트 비움
                    comp.clear()
                
            # 그래프 출력용
            if np.real(cor_P) >= 0:
                demod_result.append(1)
            else:
                demod_result.append(-1)
           
    except IndexError or ValueError:
        print('end')
    
    return bit, demod_result, cor_result, code_delay_lst, dop_freq_lst, phase_offset_lst, phase_offset_lst_deg 

In [16]:
# 상위 5개의 SV에 대해 Tracking & Demodulation 수행

svBits = []
# 검사용(발산 안했나)
svFinal_EstFreq = []
svFinal_Estcode = []

outer = tqdm(range(5), desc = "Tracking", position = 0) 
#bar = tqdm(range(9998), desc = "Process", position = 1)

for i in outer:
    outer.set_description(f'Tracking SV {SVresult[i][0] + 1}')
    #print(f'Processing SV {SVresult[i][0] + 1}:')
    bit, dem, cr, cdl, dfl, pol, pold = Tracking(SVresult[i][0], SVresult[i][1], SVresult[i][2], SVresult[i][3])
    svBits.append(bit)
    svFinal_EstFreq.append(dfl[-1])
    svFinal_Estcode.append(cdl[-1])

print()
for s in SVresult:
    print(f"SV : {s[0]+1}, code delay offset : {s[1]}, doppler frequency offset : {int(s[2])}")
print(*svFinal_EstFreq)
print(*svFinal_Estcode)

Tracking SV 20: 100%|██████████| 5/5 [05:01<00:00, 60.36s/it]


SV : 5, code delay offset : 164.0, doppler frequency offset : 500
SV : 11, code delay offset : 628.0, doppler frequency offset : -2000
SV : 13, code delay offset : 269.0, doppler frequency offset : 1500
SV : 15, code delay offset : 83.5, doppler frequency offset : 3000
SV : 20, code delay offset : 116.0, doppler frequency offset : -500
SV : 29, code delay offset : 970.5, doppler frequency offset : -500
SV : 30, code delay offset : 844.0, doppler frequency offset : 0
311.24103498670826 -2084.3667591156313 1171.0836190205255 2839.241800756128 -633.9346025278577
161.5243399999983 641.4831399999719 261.0615199999945 64.94278000000007 119.76463999999999


In [13]:
#for b in svBits:
#    print(*b)

0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0 1 0 1 0 1 0 0 0 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 1 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 0 1 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 1 0 1 0 1 1 1 0 0 0 0 1 0 1 0 0 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0 1 0 1 0 1 0 0 0 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 0 0 1 0 1 1 0 1 1 0 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 0
1 

In [14]:
preamble = [1, 0, 0, 0, 1, 0, 1, 1]  # 10001011
preamble_inv = [1 - bit for bit in preamble]  # 01110100

# 프리앰블 검색 함수
def find_preamble_positions(bit_list, preamble, preamble_inv):
    positions = []
    bit_length = len(bit_list)
    preamble_length = len(preamble)
    phase = None  # 최초 프리앰블의 위상 (None: 아직 결정되지 않음)

    i = 0
    while i <= bit_length - preamble_length:
        # 현재 위치에서 시작하는 8비트 추출
        segment = bit_list[i:i + preamble_length]

        # 프리앰블 패턴과 비교
        if phase is None:
            # 최초 프리앰블 검색
            if segment == preamble:
                phase = 'normal'
                positions.append(i)
                i += preamble_length  # 프리앰블 길이만큼 이동
            elif segment == preamble_inv:
                phase = 'inverted'
                positions.append(i)
                i += preamble_length  # 프리앰블 길이만큼 이동
            else:
                i += 1  # 다음 비트로 이동
        else:
            # 최초 프리앰블 이후
            if phase == 'normal' and segment == preamble:
                positions.append(i)
                i += preamble_length  # 프리앰블 길이만큼 이동
            elif phase == 'inverted' and segment == preamble_inv:
                positions.append(i)
                i += preamble_length  # 프리앰블 길이만큼 이동
            else:
                i += 1  # 다음 비트로 이동

    return positions

# 프리앰블 위치 찾기
svPreamble_Loc = []
for b in svBits:
    positions = find_preamble_positions(b, preamble, preamble_inv)
    svPreamble_Loc.append(positions)
# 결과 출력
for i in range(5):
    print(f"SV {SVresult[i][0]+1}의 프리앰블이 발견된 위치: {svPreamble_Loc[i]}")
    

SV 5의 프리앰블이 발견된 위치: [19, 40, 101, 319, 340]
SV 11의 프리앰블이 발견된 위치: [19, 40, 297, 319, 340, 475]
SV 13의 프리앰블이 발견된 위치: [9, 19, 40, 107, 199, 319, 340, 396]
SV 15의 프리앰블이 발견된 위치: [19, 40, 319, 340]
SV 20의 프리앰블이 발견된 위치: [19, 40, 97, 262, 319, 340]
